In [1]:
import numpy as np
import cv2

In [2]:
with_mask = np.load('Task2_with_mask.npy')
without_mask = np.load('Task2_without_mask.npy')

In [3]:
print(with_mask.shape)
print(without_mask.shape)

(200, 50, 50, 3)
(200, 50, 50, 3)


In [4]:
with_mask = with_mask.reshape(200,50* 50 * 3)  
without_mask=without_mask.reshape(200,50* 50 * 3)

In [5]:
print(with_mask.shape)
print(without_mask.shape)

(200, 7500)
(200, 7500)


In [6]:
X = np.r_[with_mask, without_mask]

In [7]:
X.shape 

(400, 7500)

In [8]:
labels = np.zeros(X.shape[0])

In [9]:
labels[200:] = 1.0

In [10]:
names = {0: 'Mask', 1:'No Mask'}

In [11]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [12]:
x_train, x_test, y_train, y_test = train_test_split(X,labels, test_size= 0.2, random_state=100)

In [13]:
x_train.shape

(320, 7500)

In [14]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3) 
x_train = pca.fit_transform(x_train)

In [15]:
x_train[0]

array([-1704.52363667,  1023.33098295,  -342.49078338])

In [16]:
x_train.shape

(320, 3)

In [17]:
svm = SVC()
svm.fit(x_train, y_train)

SVC()

In [18]:
x_test = pca.transform(x_test)
y_pred = svm.predict(x_test)

In [19]:
y_pred

array([1., 1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1.,
       1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1.,
       1., 1., 0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 0.,
       1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1.,
       0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0.])

In [20]:
haar_data = cv2.CascadeClassifier('data.xml')

In [ ]:
capture = cv2.VideoCapture(0) 
data = []
font = cv2.FONT_HERSHEY_SIMPLEX
while True:
    flag, img = capture.read()
    if flag: 
        faces = haar_data.detectMultiScale(img)
        for x,y,w,h in faces:
            cv2.rectangle(img, (x,y), (x+w, y+h), (255,0,255), 4)
            face = img[y:y+h , x:x+w, :] 
            face =cv2.resize(face, (50,50)) 
            face = face.reshape(1, -1)
            face = pca.transform(face)
            pred = svm.predict(face)[0]
            n = names[int(pred)]
            cv2.putText(img, n, (x,y), font, 1, (244,250,255), 3)
            print(n)
        cv2.imshow('result',img)
        if cv2.waitKey(2) == 27:
            break
capture.release() 
cv2.destroyAllWindows() 